In [13]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.cluster import KMeans 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.metrics import pairwise_distances

import nltk
nltk.download('stopwords')
nltk.download('punkt')
import string

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')


# email module has some useful functions
import os, sys, email,re


[nltk_data] Downloading package stopwords to /home/ki/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ki/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [119]:
df = pd.read_csv('projectfinder.csv')
df.dropna(axis=0, inplace=True)
df.head()
df.describe()

,skill_summary,title,description
count,5360,5360,5360
unique,5083,4979,5282
top,"dokumentation, 2nd level support, windows, sup...",Technischer Support,Projektbeschreibung \n\n ...
freq,17,23,19


In [120]:
title = df['title']
description = df['description'].tolist()
skill = df['skill_summary']
print(type(description))

<class 'list'>


In [121]:
# load nltk's German stopwords'
with open('stopwords-de.txt', 'r') as f:
    stopwords = f.read().splitlines()
print(len(stopwords))
stopwords_eng = nltk.corpus.stopwords.words('english')
stopwords.extend(stopwords_eng)
print(len(stopwords))

622
801


In [122]:
# load nltk's SnowballStemmer
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("german")

In [123]:
#here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    #convert text list from list to a string
    text_to_string = ','.join(str(v) for v in text)
    
    # sentence tokenization and lowercasing first, then by word to ensure that punctuation is caught as it's own token
    list_of_tokens = [word.lower() for sentence in nltk.sent_tokenize(text_to_string) for word in nltk.word_tokenize(sentence)]
    
    #extract only alphanumic tokens and add to cleaned_tokens
    alphanumeric_tokens = [token for token in list_of_tokens if token.isalnum()]
    print(alphanumeric_tokens[:20])
    
    #stopwords removal
    cleaned = [word for word in alphanumeric_tokens if word not in stopwords]
    print(cleaned[:20])
    
    #stemming
    stemmed_tokens = [stemmer.stem(token) for token in cleaned]
    print(stemmed_tokens[:20])

    return stemmed_tokens

# this is used only for presentational purposes
def tokenize_only(text):
    #convert text list from list to a string
    text_to_string = ','.join(str(v) for v in text)
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    list_of_tokens = [word.lower() for sent in nltk.sent_tokenize(text_to_string) for word in nltk.word_tokenize(sent)]
    
    #stopwords removal
    cleaned = [word for word in list_of_tokens if word not in stopwords]
    
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    filtered_tokens = [token for token in cleaned if token.isalnum()]
    

    return filtered_tokens

In [124]:
stemmed_text = tokenize_and_stem(description)
tokenized_text = tokenize_only(description)
#for word in description:
 #   stemmed_word = tokenize_and_stem(word) #for each item in 'synopses', tokenize/stem
  #  all_tokenized = tokenize_only(word)
   # stemmed_text.extend(stemmed_word) #extend the 'totalvocab_stemmed' list
    #tokenized_text.extend(all_tokenized)
    


['business', 'intelligence', 'analyst', 'tableau', 'desktop', 'für', 'ein', 'kundenprojekt', 'suchen', 'wir', 'sie', 'als', 'business', 'intelligence', 'analyst', 'tableau', 'desktop', 'der', 'kunde', 'hat']
['business', 'intelligence', 'analyst', 'tableau', 'desktop', 'kundenprojekt', 'suchen', 'business', 'intelligence', 'analyst', 'tableau', 'desktop', 'kunde', 'hohes', 'kontinui', 'konzeption', 'customizing', 'softwareanpassungen', 'talend', 'esb']
['business', 'intelligenc', 'analyst', 'tableau', 'desktop', 'kundenprojekt', 'such', 'business', 'intelligenc', 'analyst', 'tableau', 'desktop', 'kund', 'hoh', 'kontinui', 'konzeption', 'customizing', 'softwareanpass', 'talend', 'esb']


In [125]:
vocab_frame = pd.DataFrame({'words': tokenized_text}, index = stemmed_text)
print('there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')

there are 562931 items in vocab_frame


In [126]:
vocab_frame.head()

,words
business,business
intelligenc,intelligence
analyst,analyst
tableau,tableau
desktop,desktop


In [139]:
vectorizer = TfidfVectorizer(stop_words=stopwords)

X = vectorizer.fit_transform(description)
X

/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['wahr'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


<5360x31978 sparse matrix of type '<class 'numpy.float64'>'
	with 490000 stored elements in Compressed Sparse Row format>

In [140]:
true_k = 5
model = KMeans(n_clusters=true_k)
model.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [141]:
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

In [143]:
for i in range(true_k):
    print(f'Cluster {i}')
    print()
    for ind in order_centroids[i, :10]:
        print(terms[ind])

Cluster 0

sap
bewerben
abap
direkt
melden
berater
fi
erfahrung
kenntnisse
hana
Cluster 1

contractor
https
impressum
www
de
münchen
informationen
frankfurt
main
betroffenenrechte
Cluster 2

hays
kontakt
passende
bewerben
positionen
profitieren
kostenfrei
erfahrung
vorteile
aufgaben
Cluster 3

experience
bewerben
knowledge
project
development
data
skills
sap
looking
working
Cluster 4

bewerben
direkt
melden
kenntnisse
erfahrung
java
entwicklung
kunden
projekt
entwickler


In [136]:

print("Prediction")
X = vectorizer.transform(["Produktumfeld der Firma VMWARE: Airwatch (sehr gute Kenntnisse)Netzwerktechnik: LAN, DMZ, Rechenzentrum (Vertiefte Kenntnisse) MS-Office Tools: Word, Excel, Visio, Powerpoint (Vertiefte Kenntnisse) Bereitschaft zur Sicherheitsüberprüfung Level 2 (Ü2)"])
predicted = model.predict(X)
print(predicted)

Prediction
[1]


In [127]:
from sklearn.feature_extraction.text import TfidfVectorizer
 
#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words=stopwords,
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(description) #fit the vectorizer to description

print(tfidf_matrix.shape)

['w', 'e', 'l', 'c', 'h', 'e', 'm']
[]
[]
['d', 'e', 'r', 'e', 'n']
[]
[]
['d', 'e', 'n', 'e', 'n']
[]
[]
['d', 'i', 'e']
[]
[]
['b', 'y']
[]
[]
['n', 'o', 'r']
[]
[]
['k', 'e', 'i', 'n', 'e', 'n']
[]
[]
['d', 'e', 's', 's', 'e', 'l', 'b', 'e', 'n']
[]
[]
['t', 'h', 'e', 'y']
[]
[]
['d', 'u', 'r', 'f', 't', 'e', 'n']
[]
[]
['n', 'e', 'u', 'n', 't', 'e']
[]
[]
['m', 'u', 's', 't', 'n']
[]
[]
['j', 'e', 'n', 'e', 's']
[]
[]
['h', 'a', 't', 't', 'e', 'n']
[]
[]
['e', 'i', 'n', 'e', 'm']
[]
[]
['s', 'e', 'i', 'n', 'e', 'm']
[]
[]
['h', 'e', 'i', 's', 's', 't']
[]
[]
['v', 'e']
[]
[]
['j', 'e', 'm', 'a', 'n', 'd']
[]
[]
['e', 'h', 'r', 'l', 'i', 'c', 'h']
[]
[]
['m', 'a', 'g', 's', 't']
[]
[]
['d', 'a', 'n', 'e', 'b', 'e', 'n']
[]
[]
['s', 'a', 'g', 't']
[]
[]
['s', 'o', 'l', 'l']
[]
[]
['s', 'o', 'l', 'c', 'h', 'e', 'r']
[]
[]
['w', 'o', 'r', 'd', 'e', 'n']
[]
[]
['d', 'e', 'm', 'g', 'e', 'm', 'ä', 'ß']
['ä', 'ß']
['a', 'ss']
['d', 'a', 'n', 'k']
[]
[]
['h', 'i', 'm', 's', 'e', 'l', 'f']
[

['m', 'i', 't', 't', 'e', 'l']
[]
[]
['n', 'u', 'r']
[]
[]
['r', 'e', 'c', 'h', 't', 'e', 's']
[]
[]
['t', 'h', 'e']
[]
[]
['r', 'e', 'c', 'h', 't', 'e', 'r']
[]
[]
['s', 'h', 'a', 'n']
[]
[]
['a', 'n', 'd', 'e', 'r']
[]
[]
['d', 'o', 'e', 's', 'n', 't']
[]
[]
['b', 'a', 'l', 'd']
[]
[]
['s', 'o', 'w', 'i', 'e']
[]
[]
['w', 'a', 'h', 'r']
[]
[]
['e', 'i', 'n', 'i', 'g', 'e', 's']
[]
[]
['z', 'u', 's', 'a', 'm', 'm', 'e', 'n']
[]
[]
['a', 'l', 'l', 'g', 'e', 'm', 'e', 'i', 'n', 'e', 'n']
[]
[]
['d', 'i', 'e', 'j', 'e', 'n', 'i', 'g', 'e', 'n']
[]
[]
['t', 'h', 'e', 'm', 's', 'e', 'l', 'v', 'e', 's']
[]
[]
['d', 'a', 'b', 'e', 'i']
[]
[]
['s', 'o', 'w', 'e', 'i', 't']
[]
[]
['a', 'c', 'h', 't']
[]
[]
['d', 'a', 'r', 'a', 'u', 'f']
[]
[]
['s', 'o', 'n', 'd', 'e', 'r', 'n']
[]
[]
['w', 'ä', 'h', 'r', 'e', 'n', 'd']
['ä']
['a']
['d', 'e', 'm', 'g', 'e', 'g', 'e', 'n', 'ü', 'b', 'e', 'r']
['ü']
['u']
['j', 'e', 'n', 'e', 'n']
[]
[]
['h', 'a', 't', 't', 'e', 't']
[]
[]
['d', 'r', 'i', 't', 't

/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ss'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


['ü', 'b', 'e', 'r', 'u', 'n', 's', 'e', 'r', 'e', 'n', 'k', 'u', 'n', 'd', 'e', 'n', 'u', 'n', 's']
['ü', 'ü', 'ü', 'ö', 'ä', 'ü', 'ü', 'ä', 'ü', 'ü', 'ä', 'ä', 'ö', 'ä', 'ä', 'ä', 'ü', 'ü', 'ö', 'ö']
['u', 'u', 'u', 'o', 'a', 'u', 'u', 'a', 'u', 'u', 'a', 'a', 'o', 'a', 'a', 'a', 'u', 'u', 'o', 'o']
['d', 'i', 'g', 'i', 't', 'a', 'l', 'i', 's', 'i', 'e', 'r', 'u', 'n', 'g', 'i', 'n', 'd', 'e', 'r']
['ü', 'ä']
['u', 'a']
['ü', 'b', 'e', 'r', 'u', 'n', 's', 'e', 'r', 'e', 'n', 'k', 'u', 'n', 'd', 'e', 'n', 'u', 'n', 's']
['ü', 'ß', 'ü', 'ä', 'ä', 'ü', 'ü', 'ü', 'ä', 'ä', 'ö', 'ä', 'ü', 'ä', 'ä', 'ä', 'ü', 'ö', 'ö']
['u', 'ss', 'u', 'a', 'a', 'u', 'u', 'u', 'a', 'a', 'o', 'a', 'u', 'a', 'a', 'a', 'u', 'o', 'o']
['i', 'h', 'r', 'e', 'a', 'u', 'f', 'g', 'a', 'b', 'e', 'n', 'm', 'i', 't', 'a', 'r', 'b', 'e', 'i']
['ä', 'ä', 'ä', 'ä', 'ö', 'ö']
['a', 'a', 'a', 'a', 'o', 'o']
['ü', 'b', 'e', 'r', 'u', 'n', 's', 'e', 'r', 'e', 'n', 'k', 'u', 'n', 'd', 'e', 'n', 'f', 'ü', 'r']
['ü', 'ü', 'ü', 

['ü', 'ä', 'ß', 'ü', 'ü', 'ü', 'ü', 'ä', 'ä', 'ä', 'ö', 'ö']
['u', 'a', 'ss', 'u', 'u', 'u', 'u', 'a', 'a', 'a', 'o', 'o']
['ü', 'b', 'e', 'r', 'u', 'n', 's', 'e', 'r', 'e', 'n', 'k', 'u', 'n', 'd', 'e', 'n', 'u', 'n', 's']
['ü', 'ö', 'ö', 'ä', 'ü', 'ä', 'ü', 'ü', 'ä', 'ä', 'ä', 'ö', 'ö']
['u', 'o', 'o', 'a', 'u', 'a', 'u', 'u', 'a', 'a', 'a', 'o', 'o']
['o', 'u', 'r', 'c', 'o', 's', 't', 'u', 'm', 'e', 'r', 'o', 'u', 'r', 'c', 'o', 's', 't', 'u', 'm']
[]
[]
['o', 'u', 'r', 'c', 'o', 's', 't', 'u', 'm', 'e', 'r', 'o', 'u', 'r', 'c', 'o', 's', 't', 'u', 'm']
[]
[]
['ü', 'b', 'e', 'r', 'u', 'n', 's', 'e', 'r', 'e', 'n', 'k', 'u', 'n', 'd', 'e', 'n', 'u', 'n', 's']
['ü', 'ä', 'ö', 'ä', 'ü', 'ü', 'ä', 'ö', 'ü', 'ö', 'ä', 'ü', 'ä', 'ß', 'ä', 'ä', 'ä', 'ö', 'ö', 'ä']
['u', 'a', 'o', 'a', 'u', 'u', 'a', 'o', 'u', 'o', 'a', 'u', 'a', 'ss', 'a', 'a', 'a', 'o', 'o', 'a']
['a', 'b', 'o', 'u', 't', 'o', 'u', 'r', 'c', 'u', 's', 't', 'o', 'm', 'e', 'r', 'o', 'u', 'r', 'c']
[]
[]
['ü', 'b', 'e', 'r'

...

['p', 'r', 'o', 'j', 'e', 'k', 't', 'b', 'e', 's', 'c', 'h', 'r', 'e', 'i', 'b', 'u', 'n', 'g', 'm']
['ü', 'ö', 'ü', 'ü', 'ü', 'ü', 'ä', 'ü', 'ü', 'ä', 'ä', 'ü', 'ö', 'ä', '½', 'ä', 'ä', 'ü', 'ö', 'ü']
['u', 'o', 'u', 'u', 'u', 'u', 'a', 'u', 'u', 'a', 'a', 'u', 'o', 'a', '½', 'a', 'a', 'u', 'o', 'u']
['p', 'r', 'o', 'j', 'e', 'k', 't', 'b', 'e', 's', 'c', 'h', 'r', 'e', 'i', 'b', 'u', 'n', 'g', 'm']
['ü', 'ö', 'ö', 'ä', 'ü', 'ä', 'ä', 'ä', 'ü']
['u', 'o', 'o', 'a', 'u', 'a', 'a', 'a', 'u']
['p', 'r', 'o', 'j', 'e', 'k', 't', 'b', 'e', 's', 'c', 'h', 'r', 'e', 'i', 'b', 'u', 'n', 'g', 'm']
['ü', 'ö', 'ö', 'ä', 'ü', 'ü', 'ä', 'ä', 'ä', 'ü']
['u', 'o', 'o', 'a', 'u', 'u', 'a', 'a', 'a', 'u']
['p', 'r', 'o', 'j', 'e', 'k', 't', 'b', 'e', 's', 'c', 'h', 'r', 'e', 'i', 'b', 'u', 'n', 'g', 'm']
['ü', 'ü', 'ä', 'ä', 'ä', 'ü']
['u', 'u', 'a', 'a', 'a', 'u']
['p', 'r', 'o', 'j', 'e', 'k', 't', 'b', 'e', 's', 'c', 'h', 'r', 'e', 'i', 'b', 'u', 'n', 'g', 'm']
['ü', 'ö', 'ü', 'ä', 'ä', 'ä', 'ä', '

['p', 'r', 'o', 'j', 'e', 'k', 't', 'b', 'e', 's', 'c', 'h', 'r', 'e', 'i', 'b', 'u', 'n', 'g', 'm']
[]
[]
['p', 'r', 'o', 'j', 'e', 'k', 't', 'b', 'e', 's', 'c', 'h', 'r', 'e', 'i', 'b', 'u', 'n', 'g', 'm']
['ü']
['u']
['p', 'r', 'o', 'j', 'e', 'k', 't', 'b', 'e', 's', 'c', 'h', 'r', 'e', 'i', 'b', 'u', 'n', 'g', 'm']
[]
[]
['p', 'r', 'o', 'j', 'e', 'k', 't', 'b', 'e', 's', 'c', 'h', 'r', 'e', 'i', 'b', 'u', 'n', 'g', 'm']
['ä', 'ü', 'ä', 'ö', 'ä', 'ä', 'ö', 'ü', 'ü', 'ü', 'ä', 'ö', 'ä', 'ä', 'ü', 'ü', 'ä', 'é', 'ü']
['a', 'u', 'a', 'o', 'a', 'a', 'o', 'u', 'u', 'u', 'a', 'o', 'a', 'a', 'u', 'u', 'a', 'é', 'u']
['p', 'r', 'o', 'j', 'e', 'k', 't', 'b', 'e', 's', 'c', 'h', 'r', 'e', 'i', 'b', 'u', 'n', 'g', 'm']
['ä', 'ü', 'ä', 'ö', 'ä', 'ß', 'ö', 'ö', 'ö', 'ö', 'ä', 'ü', 'ü', 'ä', 'ä', 'ü', 'ä', 'ü', 'ü', 'ä']
['a', 'u', 'a', 'o', 'a', 'ss', 'o', 'o', 'o', 'o', 'a', 'u', 'u', 'a', 'a', 'u', 'a', 'u', 'u', 'a']
['p', 'r', 'o', 'j', 'e', 'k', 't', 'b', 'e', 's', 'c', 'h', 'r', 'e', 'i', 

['p', 'r', 'o', 'j', 'e', 'k', 't', 'b', 'e', 's', 'c', 'h', 'r', 'e', 'i', 'b', 'u', 'n', 'g', 'm']
['ä', 'ü', 'ä', 'ä', 'ä', 'ü', 'ü', 'ö', 'ü', 'ä', 'ö', 'ü']
['a', 'u', 'a', 'a', 'a', 'u', 'u', 'o', 'u', 'a', 'o', 'u']
['p', 'r', 'o', 'j', 'e', 'k', 't', 'b', 'e', 's', 'c', 'h', 'r', 'e', 'i', 'b', 'u', 'n', 'g', 'm']
['ü', 'ä', 'ä', 'ä', 'ß', 'ü']
['u', 'a', 'a', 'a', 'ss', 'u']
['p', 'r', 'o', 'j', 'e', 'k', 't', 'b', 'e', 's', 'c', 'h', 'r', 'e', 'i', 'b', 'u', 'n', 'g', 'm']
['ü', 'ä', 'ü', 'ü', 'ä', 'ü', 'ü', 'ä', 'ü', 'ü', 'ä', 'ö', 'ü', 'ö']
['u', 'a', 'u', 'u', 'a', 'u', 'u', 'a', 'u', 'u', 'a', 'o', 'u', 'o']
['p', 'r', 'o', 'j', 'e', 'k', 't', 'b', 'e', 's', 'c', 'h', 'r', 'e', 'i', 'b', 'u', 'n', 'g', 'm']
['ä', 'ä', 'ä', 'ä', 'ü', 'ä', 'ü', 'ä', 'ß', 'ü', 'ä', 'ä', 'ä', 'ä', 'ä', 'ä', 'ü']
['a', 'a', 'a', 'a', 'u', 'a', 'u', 'a', 'ss', 'u', 'a', 'a', 'a', 'a', 'a', 'a', 'u']
['p', 'r', 'o', 'j', 'e', 'k', 't', 'b', 'e', 's', 'c', 'h', 'r', 'e', 'i', 'b', 'u', 'n', 'g', 

['ü', 'ä', 'ü', 'ü', 'ü', 'ä', 'ö', 'ä', 'ß', 'ä', 'ü', 'ä', 'ü', 'ü', 'ü', 'ß']
['u', 'a', 'u', 'u', 'u', 'a', 'o', 'a', 'ss', 'a', 'u', 'a', 'u', 'u', 'u', 'ss']
['p', 'r', 'o', 'j', 'e', 'k', 't', 'b', 'e', 's', 'c', 'h', 'r', 'e', 'i', 'b', 'u', 'n', 'g', 'm']
['ü']
['u']
['p', 'r', 'o', 'j', 'e', 'k', 't', 'b', 'e', 's', 'c', 'h', 'r', 'e', 'i', 'b', 'u', 'n', 'g', 'm']
['ü', 'ü', 'ü', 'ü', 'ö', 'ß', 'ü', 'ä', 'ö', 'ä', 'ü', 'ö', 'ü', 'ä', 'ü', 'ä', 'ä', 'ä', 'ü', 'ü']
['u', 'u', 'u', 'u', 'o', 'ss', 'u', 'a', 'o', 'a', 'u', 'o', 'u', 'a', 'u', 'a', 'a', 'a', 'u', 'u']
['p', 'r', 'o', 'j', 'e', 'k', 't', 'b', 'e', 's', 'c', 'h', 'r', 'e', 'i', 'b', 'u', 'n', 'g', 'm']
['ä', 'ü', 'ö', 'ä', 'ö', 'ä']
['a', 'u', 'o', 'a', 'o', 'a']
['p', 'r', 'o', 'j', 'e', 'k', 't', 'b', 'e', 's', 'c', 'h', 'r', 'e', 'i', 'b', 'u', 'n', 'g', 'm']
['ü', 'ü', 'ö', 'ü', 'ü', 'ü', 'ü', 'ö', 'ä', 'ä', 'ß', 'ö', 'ö', 'ü', 'ü', 'ä', 'ä', 'ä', 'ä', 'ö']
['u', 'u', 'o', 'u', 'u', 'u', 'u', 'o', 'a', 'a', 'ss

[]
[]
['p', 'r', 'o', 'j', 'e', 'k', 't', 'b', 'e', 's', 'c', 'h', 'r', 'e', 'i', 'b', 'u', 'n', 'g', 'm']
['ü', 'ü', 'ä', 'ä', 'ü', 'ö', 'ü', 'ö', 'ä', 'ü', 'ü', 'ü', 'ä', 'ü', 'ü', 'ä', 'ü', 'ä', 'ä', 'ü']
['u', 'u', 'a', 'a', 'u', 'o', 'u', 'o', 'a', 'u', 'u', 'u', 'a', 'u', 'u', 'a', 'u', 'a', 'a', 'u']
['p', 'r', 'o', 'j', 'e', 'k', 't', 'b', 'e', 's', 'c', 'h', 'r', 'e', 'i', 'b', 'u', 'n', 'g', 'm']
['ü', 'ä', 'ß', 'ü', 'ß', 'ü', 'ä', 'ö', 'ö', 'ö', 'ä', 'ü', 'ü', 'ü', 'ü', 'ö', 'ü', 'ü', 'ß']
['u', 'a', 'ss', 'u', 'ss', 'u', 'a', 'o', 'o', 'o', 'a', 'u', 'u', 'u', 'u', 'o', 'u', 'u', 'ss']
['p', 'r', 'o', 'j', 'e', 'k', 't', 'b', 'e', 's', 'c', 'h', 'r', 'e', 'i', 'b', 'u', 'n', 'g', 'm']
['ü', 'ö', 'ü', 'ü', 'ü', 'ü', 'ö', 'ä', 'ä', 'ß', 'ö', 'ö', 'ü', 'ä', 'ä', 'ä', 'ä', 'ä', 'ß', 'ü']
['u', 'o', 'u', 'u', 'u', 'u', 'o', 'a', 'a', 'ss', 'o', 'o', 'u', 'a', 'a', 'a', 'a', 'a', 'ss', 'u']
['p', 'r', 'o', 'j', 'e', 'k', 't', 'b', 'e', 's', 'c', 'h', 'r', 'e', 'i', 'b', 'u', 'n',

In [128]:
terms = tfidf_vectorizer.get_feature_names()
terms

['ss']

In [111]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)
dist

array([[-2.22044605e-16,  1.00000000e+00,  8.64780483e-01, ...,
         7.62872459e-01,  5.47389137e-01,  5.81232584e-01],
       [ 1.00000000e+00,  0.00000000e+00,  1.00000000e+00, ...,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
       [ 8.64780483e-01,  1.00000000e+00,  0.00000000e+00, ...,
         9.67935728e-01,  8.54688701e-01,  8.19076957e-01],
       ...,
       [ 7.62872459e-01,  1.00000000e+00,  9.67935728e-01, ...,
         0.00000000e+00,  5.86348037e-01,  5.67144662e-01],
       [ 5.47389137e-01,  1.00000000e+00,  8.54688701e-01, ...,
         5.86348037e-01, -2.22044605e-16,  8.01994160e-01],
       [ 5.81232584e-01,  1.00000000e+00,  8.19076957e-01, ...,
         5.67144662e-01,  8.01994160e-01, -4.44089210e-16]])

In [112]:
from sklearn.cluster import KMeans

num_clusters = 3 

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

CPU times: user 19.4 s, sys: 260 ms, total: 19.6 s
Wall time: 19.8 s


In [113]:
from sklearn.externals import joblib

joblib.dump(km,  'doc_cluster.pkl')

km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

In [114]:
projects = {'description': description, 'cluster': clusters }

frame = pd.DataFrame(projects, index = [clusters] , columns = ['cluster', 'description'])

In [115]:
frame['cluster'].value_counts()

0    3575
1    1100
2     685
Name: cluster, dtype: int64

In [116]:
from __future__ import print_function

print("Top terms per cluster:")
print()
#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
    print() #add whitespace
    print() #add whitespace
    
    #print("Cluster %d titles:" % i, end='')
    #for title in frame.ix[i]['title'].values.tolist():
     #   print(' %s,' % title, end='')
    print() #add whitespace
    print() #add whitespace
    
print()
print()

Top terms per cluster:

Cluster 0 words:

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if sys.path[0] == '':


 b'f\xc3\xbcr', b'erfahrung', b'kenntnisse', b'entwicklung', b'gute', b'kunde',



Cluster 1 words: b'sap', b'beratung', b'f\xc3\xbcr', b'erfahrung', b'kunde', b'projektes',



Cluster 2 words: b'skills', b'teams', b'java', b'start', b'asap', b'profil',





